### 아이디어
- K가 적다. 
- 모든 숫자가 다른 숫자이면서 $1 \le A_i \le N$ 이므로, 1부터 N까지 각 숫자가 한번씩 등장한다는 것을 알 수 있다.
  - 예를들어, 1 뒤에 있는 숫자의 개수를 세어둔다면 어느정도 위치 정보도 알 수 있는 것이 아닐까?
- (관찰) max 세그트리를 만들고, L[i] 부터 L[N]까지에 대해서 쿼리를 했을때 나온값이 $v$ 라고 할 때 $v > L[i]$ 라면 L[i]로 부터 적어도 수열 1개를 만들 수 있다.
  - 이걸 어떤 숫자 j가 있는 인덱스에서 수행해서 `j보다 크면서 j보다 오른쪽에 있는 숫자가 있는지의 여부`도 알 수 있다

### 풀이
- 길이가 K인 Increasing Subsequence의 개수를 구하는 문제이다.
- 시간복잡도 $\Omicron(KN \log N)$
- $l \in L$ 이고, $k := 1 \to K$ 에 대해, $L_i$ 에서 끝나면서 길이가 $k$ 인 증가 수열의 개수를 효율적으로 세야 한다.\
그러기 위해선 길이 $k-1$ 인 수열에서 $L_i$ 보다 작은 증가 수열의 개수를 세면 된다.
- 세그먼트 트리를 이용해 길이 $k-1$ 의 증가수열 개수 리스트 $L'$ 에 대해, $\displaystyle\sum_{i=1}^{L_i-1} L'_i$ 를 효율적으로 구할 수 있다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

from operator import add
class ST:
  def __init__(self, L, f=add, default=0):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __delitem__(self, i):
    self[i] = self._def

  def __getitem__(self, i):
    return self.L[i + self._size]

  def __setitem__(self, i, v):
    i += self._size
    self.L[i] = v
    i >>= 1
    while i:
      self.L[i] = self._f(self.L[2 * i], self.L[2 * i + 1])
      i >>= 1

  def __len__(self):
    return self._len

  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s >>= 1
      e >>= 1

    return self._f(l, r)

MOD = 10**9+7
def sol() :
  N, K = map(int, input().split())
  L = [*map(int, input().split())]

  st = [ST([0] * N, f=lambda x, y: x+y % MOD) for _ in range(K + 1)]
  for i in range(N) :
    for k in range(1, K + 1) :
      if k == 1 :
        st[k][L[i]-1] = 1 # 길이1인 수열은 모두 증가수열이다.
      else :
        st[k][L[i]-1] = st[k - 1].query(0, L[i]-1) % MOD # L[i]보다 작은 수 중에서 길이 k - 1인 증가수열의 개수를 구한다.

  print(st[K].query(0, N))

sol()